In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn import datasets
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
import pandas as pd
from nltk.metrics import ConfusionMatrix
import numpy as np
from sklearn.externals import joblib

In [2]:
emo = 2

if emo == 2: 
    dataset_path = '../Data/Datasets/Binary Classification/'
else:
    dataset_path = '../Data/Datasets/Multiclass Classification/'

In [3]:
def convert_features(sentence):
    return {i:(i in sentence) for i in word_features}

In [4]:
#save the classifier based on the whole dataset
train = pd.read_csv(dataset_path+'train.csv', sep=",", header=None,index_col = False)
test = pd.read_csv(dataset_path+'test.csv', sep=",", header=None,index_col = False)

whole_ds = train.append(test,ignore_index=True )
whole_ds['tokenized_sents'] = whole_ds.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)

all_words = []
for line in whole_ds['tokenized_sents']:
    for w in line:
        all_words.append(w.lower())
all_words = nltk.FreqDist(all_words)
word_features = list(all_words.keys())
joblib.dump(word_features, '../Data/nb_word_features.pkl') 


featured_whole_ds = [(convert_features(row['tokenized_sents']), row[1]) for index, row in whole_ds.iterrows()]
classifier = nltk.NaiveBayesClassifier.train(featured_whole_ds)
joblib.dump(classifier, dataset_path+'nb_classifier.pkl') 

['../Data/Datasets/Binary Classification/nb_classifier.pkl']

In [5]:
accuracy_nb = []
folds = 7

for idx in range(1,folds+1):
    
    train = pd.read_csv(dataset_path+'cv'+str(idx)+'_train.csv', sep=",", header=None,index_col = False)
    test = pd.read_csv(dataset_path+'cv'+str(idx)+'_test.csv', sep=",", header=None,index_col = False)
    
    train['tokenized_sents'] = train.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
    featured_train = [(convert_features(row['tokenized_sents']), row[1]) for index, row in train.iterrows()]
    test['tokenized_sents'] = test.apply(lambda row: nltk.word_tokenize(row[0]), axis=1)
    featured_test = [(convert_features(row['tokenized_sents']), row[1]) for index, row in test.iterrows()]

    classifier = nltk.NaiveBayesClassifier.train(featured_train)
    score = nltk.classify.accuracy(classifier, featured_test)
    accuracy_nb.append(score)
    print(score)

0.9365671641791045
0.9104477611940298
0.9402985074626866
0.9067164179104478
0.9213483146067416
0.8670411985018727
0.7921348314606742


In [6]:
pred = np.zeros((test.shape[0],1))
for index, row in test.iterrows():
    featured_item = (convert_features(row['tokenized_sents']))
    pred[index]  = int(classifier.classify(featured_item))
test['prediction'] = pred

confusion_matrix = np.zeros((5,5))
for index, row in test.iterrows():
    ind_i = row[1]
    ind_j = int(row['prediction'])
    confusion_matrix[ind_i][ind_j] = confusion_matrix[ind_i][ind_j] + 1
        
print(confusion_matrix)

accuracy = (confusion_matrix[1,1]+confusion_matrix[2,2]+confusion_matrix[0,0]+confusion_matrix[4,4]+confusion_matrix[3,3])/test.shape[0]
print(accuracy)


[[ 71.  76.   0.   0.   0.]
 [ 35. 352.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.]]
0.7921348314606742
